# Preprocessing text-mining
Ce notebook a pour objectif le traitement des textes obtenus par océrisation des images.
L'objectif est donc d'obtenir, pour chaque image (=document) un texte amélioré, permettant d'alimenter des modèles de Machine learning.

La base de texte choisie est l'ocr collectif: comme cela a été vu procédément, il contient le plus grand nombre de données (environ 398k documents sur les 400k possibles) et les informations contenues semblent identiques à celles des documents individuels).

Dans ce notebook, on va donc créer un dataframe qui aura ce format (les colonnes ocr_x sont des colonnes obtenues après traitement de raw_ocr):

| document_id (index) | raw_ocr | ocr_1 |...| ocr_n | label 

En fin de notebook, il sera ainsi possible de créer les Dataframes X (ocr_x) et y (label) qui permettront d'alimenter les modèles.

La gestion du test_train_split est faite en début de notebook, en respectant la répartition définie dans le dataset RVL-CDIP.

Remarque: des contraintes temporelles (temps alloué au projet + temps de calculs requis) nous ont contraints à nous limiter à un seul jeu de données traitées ("ocr_1"). Nous avons toutefois laissé la structure imaginée pour montrer la démarche dans laquelle nous nous inscrivions.

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import re
import os
import time
import jamspell
import pandas as pd

In [ ]:
df = pd.read_parquet(PATHS.processed_data / "df_raw_ocr.parquet")

# 2. Application des traitements

## 3.1. Création ocr_1
Ce premier pipeline de traitement permettra d'arriver à la création de la colonne ocr_1

### 3.1.1. Suppression des pagesNbr

In [ ]:
df = df[:10]

In [ ]:
pg_regex = re.compile(r'pgNbr=[0-9]+')

In [ ]:
def count_pgNbr(text):
    return len(pg_regex.findall(text))
# comptabilisation du nombre de pgNbr:

def remove_pgNbr(text):
    text = pg_regex.sub('', text)
    return text

avant = df.raw_ocr.apply(count_pgNbr).sum()
df["ocr_tmp"] = df.raw_ocr.apply(remove_pgNbr)
apres = df.ocr_tmp.apply(count_pgNbr).sum()
print(avant, apres)

### 3.1.2. Déséchappement html

In [ ]:
import html
def unescape_html(text):
    if not text:
        return text
    return html.unescape(text)

avant = df.ocr_tmp.str.contains("&lt;").sum()
df.loc[:,"ocr_tmp"] = df.ocr_tmp.apply(unescape_html)
apres = df.ocr_tmp.str.contains("&lt;").sum()
avant, apres

### 3.1.3. Correction OCR
Nous allons pour cela utiliser l'outil jamspell

#### <span style="color:red">Cette partie est fonctionnelle mais a été désactivée pour des raisons de performance</span>
(3' sur un échantillon de 1000 lignes ==> environ 20 heures, si la croissance est linéaire)

### 3.1.4 Suppression des caractères spéciaux et des séquences ne correspondant pas à des informations

In [ ]:
import re

# A améliorer pour prendre en compte les lignes
# sans doute trop brutal / il faudra le réviser sur d'autres versions ultérieures?
def basic_word_filter(text):
    if not text:
        return text
    text = text.lower()
    # Attention, c'est brutal, ca supprime tous les chiffres aussi...
    word_regex = re.compile(r'[a-z]{2,}')
    text = ' '.join(word_regex.findall(text))
    
    return text
df["ocr_tmp"] = df.ocr_tmp.apply(basic_word_filter)

### 3.1.5. Filtrage des stop_words

In [ ]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('punkt_tab')
# Télécharger les ressources NLTK nécessaires
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
#Liste des stop words en anglais
stop_words = set(stopwords.words('english'))

# Fonction pour nettoyer une phrase
def remove_stopwords(text):
    if pd.isnull(text):  # gestion des valeurs manquantes
        return ""
    words = word_tokenize(text.lower())
    filtered = [word for word in words if word.isalpha() and word not in stop_words]
    return " ".join(filtered)

# Application sur la colonne raw_ocr
df['raw_ocr_clean'] = df['ocr_tmp'].apply(remove_stopwords)

# Affichage
print(df[['ocr_tmp', 'raw_ocr_clean']])


### 3.1.6. Application du Pipeline et création d'ocr_1

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import pandas as pd
from src.models.preprocessing import text_preprocessing1
from p_tqdm import p_map

df = pd.read_parquet(PATHS.processed_data / "df_raw_ocr.parquet")

In [ ]:
# Version monothread / tres lente
ocr1 = text_preprocessing1(df.raw_ocr, with_tqdm=True)

In [ ]:
df_ocr1 = pd.DataFrame(ocr1, columns=["ocr"], index = df.index)
df_ocr1.to_parquet(PATHS.processed_data / "df_txt_ocr1.parquet")